双子実験経路生成配分コードver1230

In [1]:
import pandas as pd
import os
import datetime
import numpy as np
from datetime import timedelta 
import csv
import time
import math
from scipy.stats import norm
from scipy.optimize import minimize
import random

In [3]:
# reading data
df_ble = pd.read_csv('/Users/takahiromatsunaga/res2023/futago/df_ble.csv')
df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/futago/df_node.csv')
df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/futago/df_link.csv')
df_demand = pd.read_csv('/Users/takahiromatsunaga/res2023/futago/df_demand.csv')

L = len(df_link)
OD = len(df_demand)

In [4]:
####### リンク接続行列 ####### ### 逆戻りと滞在ありの経路
def Imaking(link_data): 
    n = len(link_data)
    I = np.eye(n) # 滞在
    for i in range(n):
        O = link_data.loc[i, 'o'] 
        D = link_data.loc[i, 'd'] 
        for j in range(n):
            if ((link_data.loc[j, 'o'] == O) or (link_data.loc[j, 'o'] == D)) or (link_data.loc[j, 'd'] == O) or (link_data.loc[j, 'd'] == D): 
                I[i, j] = 1
    return(I)

I = Imaking(df_link)

## スタートリンク25と吸収リンク26だけ操作
# リンク25(index24)へはどこからもいかない（スタート→スタートもない）
I[:, 24] = 0

# リンク26(index25)からは26にしかいかない
I[25, :] = 0
I[25, 25] = 1

# 吸収はリンク26（index25, 仮想ノード17(100, 100, 0)に接続）のみ
ddata = [26]
D = len(ddata)

In [32]:
## param init
assumed_x = np.array((-2, 1.5, 1)) # CC, DC, beta

## 観測パラメータ
sigma = 5 ## 小さいほど，生成されるd_estがdに近づく（それはそう）
N = 2
rssi_max = -40 ## これは固定して，この値は超えないように観測分布も与えるようにする

# 最短でT=7でスタートリンクから吸収リンクまで到達．
T = 10 # 試しに10で

In [14]:
##########################
###### 時空間プリズム ######
########################## 
def TSNW(ODlist):
    Ilist = []

    # 起点Oからの到達可能性指示変数io
    Itlist = np.zeros((T, L, L)) ## これは各状態に対して与えるので遷移回数T-1よりひとつ多くてT成分（timestep数がTなら状態数はT）
    II = np.copy(I)
    Itlist[0, :, :] = np.eye(L) 

    for ts in range(1, T): 
        Itlist[ts, :, :] = II 
        II = np.dot(II, I)
        II = np.where(II > 0, 1, 0) 

    # 終点Dからの到達可能性指示変数id 
    Ittlist = np.zeros((T, L, L))
    for ts in range(T): 
        Ittlist[ts, :, :] = np.transpose(Itlist[T - ts - 1, :, :]) 

    OD = len(ODlist)
    for od in range(OD):   
        ao = ODlist.loc[od, 'o'] - 1  
        aabs = ODlist.loc[od, 'd'] - 1  

        Id = np.zeros((T-1, L, L)) # Id[t]は時刻tの時の利用可能遷移k→aを示す．最後のt=T-1のとき不要なので要素数はt=0~T-2のT-1個

        for ts in range(T-1): # Tまでに吸収されるのでdはaabs．遷移回数はT-1なのでこれでOK
            if ts == 0:
                Id[ts, ao, :] = I[ao, :] 
                continue
            
            alist = np.where(Itlist[ts + 1, ao, :] == Ittlist[ts + 1, aabs, :])[0] ## ts=1（つまり第二回目の状態）のとき3番目の状態を見ている→OK
            ## Itlistの長さはT．最後はT-2で，そのときT-2+1=T-1, 
            for a in alist:
                if Itlist[ts + 1, ao, a] == Ittlist[ts + 1, aabs, a]: # always True
                    klist = np.where(I[:, a] == 1)[0]
                    
                    for k in klist:
                        if len(np.where(Id[ts - 1, :, k] == 1)[0]) != 0:
                            Id[ts, k, a] = 1 

        Ilist.append(Id)
    
    return Ilist


In [8]:
#######################
######### 準備 ######## 
####################### # 混雑による減衰は与えてない．NWごとに分散変えるというのはありか．

###### 各linkのoとdの座標を入れておく配列 ###### 
link_loc_array = [] 
for i in range(len(df_link)):
    linkid = df_link.loc[i, 'linkid']
    O = df_link.loc[i, 'o']
    D = df_link.loc[i, 'd']

    x_o = df_node[df_node['nodeid'] == O]['x'].iloc[0]
    y_o = df_node[df_node['nodeid'] == O]['y'].iloc[0]
    z_o = df_node[df_node['nodeid'] == O]['z'].iloc[0] 
    o_loc = np.array([x_o, y_o, z_o])

    x_d = df_node[df_node['nodeid'] == D]['x'].iloc[0]
    y_d = df_node[df_node['nodeid'] == D]['y'].iloc[0]
    z_d = df_node[df_node['nodeid'] == D]['z'].iloc[0]
    d_loc = np.array([x_d, y_d, z_d])

    loc_tuple = (o_loc, d_loc)
    link_loc_array.append(loc_tuple) 


###### 各ビーコンの座標を入れておく配列 ###### 
beacon_loc_array = []
for j in range(len(df_ble)):
    beacon_id = df_ble.loc[j, 'bleid']
    x_j = df_ble[df_ble['bleid'] == beacon_id]['x'].iloc[0]
    y_j = df_ble[df_ble['bleid'] == beacon_id]['y'].iloc[0]
    z_j = df_ble[df_ble['bleid'] == beacon_id]['z'].iloc[0]
    
    beacon_loc = np.array([x_j, y_j, z_j])
    beacon_loc_array.append(beacon_loc)


###### ビーコンとリンク線分の中点間距離を返す関数 ######
def dist_to_mid(p1, p2, x):
    p1 = np.array(p1)
    p2 = np.array(p2)
    x = np.array(x)

    segment = p2 - p1
    v1 = x - p1
    v2 = x - p2
    mid_point = (p1 + p2) / 2

    v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment # 垂直ベクトルを計算
    distance = np.linalg.norm(x - mid_point)
    
    return distance


###### d_arrayの用意 ###### # 各リンクの中点とbleビーコンとの距離配列
d_array = np.zeros((len(df_link), len(df_ble)))
for i in range(len(df_link)):
    p_o = link_loc_array[i][0] # o座標 # i=1の時
    p_d = link_loc_array[i][1] # d座標

    for j in range(len(df_ble)):
        x_ap = df_ble.loc[j, 'x']
        y_ap = df_ble.loc[j, 'y']
        z_ap = df_ble.loc[j, 'z']
        p_ap = np.array([x_ap, y_ap, z_ap])

        d_array[i, j] = dist_to_mid(p_o, p_d, p_ap) 

In [10]:
print(link_loc_array[0][1])

[20  0  0]


経路選択モデル部分

ちょっと丁寧に過去コード見て見直すことが必要

In [9]:
###########################
####### 経路選択モデル ######
###########################

###### 即時効用行列 ###### 
def Mset(x): 
    #Probs_T = Probs.T # 内生性バージョン
    inst = np.zeros((T-1, L, L))
    for t in range(T-1):
        inst_t = np.exp(df_link['cc'] * x[0] + df_link['dc'] * x[1])    ### + Probs_T[:, t] * x[1])    # + df_link['staire'] * x[1]) # + df_link_integrated['staire_with_esc'] * x[3])
        inst_t = pd.concat([inst_t]*L, axis=1)
        inst_t = inst_t.T
        inst_t_numpy = inst_t.values 
        inst[t, :, :] = inst_t_numpy

    return inst


###### リンク選択確率計算 ######
def newPall(x):
    Pall = np.zeros((OD, T-1, L, L)) # 個人時刻毎の遷移確率行列 遷移回数なのでT-1．今回Tを状態の数としているので遷移数はT-1になる
    beta = x[-1]

    #### 以降odごとに処理 #### 
    for od in range(OD): 
        Id = Ilist[od] 

        M = np.zeros((T-1, L, L))
        for ts in range(T-1):
            Mts = Id[ts, :, :] * Mset(x)[ts, :, :] 
            M[ts, :, :] = Mts
        
        ##### 価値関数 ##### # newV相当
        z = np.ones((T, L))
        for t in range(T-1, 0, -1):
            zii = M[t-1, :, :] * (z[t, :] ** beta) 
            zi = zii.sum(axis = 1)
            z[t-1, :] = (zi == 0) * 1 + (zi != 0) * zi

        ##### 選択確率行列 ##### 
        for t in range(T-1): 
            for k in range(L):
                for a in range(L):
                    if M[t, k, a] == 0: # 接続条件を満たせなかった観測は排除（logzero回避）
                        continue 
                    Pall[od, t, k, a] += np.exp(np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])) 

    return Pall 

In [10]:
#################
###### 配分 ######
#################

def assignment(x): 
    Pall = newPall(x)
    res_all = np.zeros(4) # columns = ['userid', 't', 'k', 'd']

    for od in range(OD): 
        ao = df_demand.loc[od, 'o'] - 1 
        ad = df_demand.loc[od, 'd'] - 1 # 吸収リンク

        # 累積確率行列
        Pi = Pall[od, :, :, :]
        for t in range(T-1): 
            for k in range(L):
                if k == 0:
                    Pi[t, :, k] = Pi[t, :, k]
                else:
                    Pi[t, :, k] += Pi[t, :, k-1]

        Nod = df_demand.loc[od, 'demand'] # 各odペアの交通量

        if Nod == 0:
            continue # odペアが存在しないならパス
        
        for nod in range(Nod): # 個人ごとに経路配分
            res_indivi = np.zeros((T, 4))
            res_indivi[:, 0] = nod          # nod(userid)
            res_indivi[:, 1] = np.arange(T) # timestep
            res_indivi[:, 3] = ad + 1       # 吸収リンクid

            ran_list = [random.random() for _ in range(T-1)] ## 乱数遷移回数T-1分発生

            kn = ao + 1                     # id
            k = ao                          # index
            res_indivi[0, 2] = kn

            for t in range(T-1): 
                ran = ran_list[t]
                for a in range(L):
                    if ran <= Pi[t, k, a]:
                        k = a
                        kn = k + 1
                        res_indivi[t+1, 2] = kn
                        break  
            
            res_all = np.vstack((res_all, res_indivi))

    res_all = np.delete(res_all, 0, axis = 0)
    df_res_all = pd.DataFrame(res_all, columns=['userid', 't', 'k', 'd'])
    df_res_all.to_csv('/Users/takahiromatsunaga/res2023/futago/assign_res_ver2.csv')

    return df_res_all

配分部分？？

In [13]:
demand_size = df_demand['demand'].sum()
Ilist = TSNW(df_demand)

x = assumed_x
df_res_all = assignment(x) # 関数内でcsv出力

#print(df_res_all)
#np.set_printoptions(threshold=np.inf)

     userid    t     k     d
0       0.0  0.0  25.0  26.0
1       0.0  1.0   1.0  26.0
2       0.0  2.0   2.0  26.0
3       0.0  3.0   3.0  26.0
4       0.0  4.0  16.0  26.0
..      ...  ...   ...   ...
995    99.0  5.0  23.0  26.0
996    99.0  6.0   9.0  26.0
997    99.0  7.0  24.0  26.0
998    99.0  8.0  26.0  26.0
999    99.0  9.0  26.0  26.0

[1000 rows x 4 columns]


In [107]:
print(newPall(x)) #### あれ25から全部のリンクに遷移可能になってるww
## 各時刻で滞在可能なリンクの絞り込みには成功してる，つまりkの絞り込みはOKだが，k→aが全部利用可能になっててそこが変
#### とりあえずPallは癌である．
#### あれ，普通にPall出力できてる

[[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00 0.00000000e+00 

In [90]:
print(Ilist[0][7]) ## t=0では利用可能リンクが1, 13だけ→これは適切に機能してる
# 1では1:1, 1:2, 1:13, 1:14, 13:1, 13:4, 13:13, 13:17が利用可能
#### Ilistは適切に機能していると見て良い．問題はPall

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 1.]
 [0.

ようやく配分終了→観測データの生成

In [33]:
## 理論的には3秒に一回の観測があるので各歩行者は一遷移20secの間に6回観測されるはずだがそれだと多い．
## 5秒に一回として，一遷移間に4回，もしくは4秒に一回として一遷移間に5回の観測とする，みたいな条件にするか．
## まあ数値実験なので何通りも試せばいいか

## 観測があるのはT=2からT=9の8timestepで8*20=160sec（T=1はOlinkにいて，T=10はDlinkにいるので，観測されるのはその間．ただT=9より前に吸収されてたらその時点で観測はないことになる．
##→観測限界からの観測プリズム的な考え方？？
### T=1, 10は架空の時間として捉えればまだ分かりが良いか．

start_time = 0
end_time = 160
m_time = 5 # 観測時間幅(s)

### とりあえず，各個人は滞在中の経路の中点において，あるいは時間帯ごとに違う点において，とにかく一つのビーコンで4回（5秒に一回なら）観測される
### さらにこれはリンクとの距離が20m以下のビーコンに対してである．→20mが適切かどうかはちょっと微妙なところがあるので，，
## とにかく，各時刻に4*2=8ログデータくらいは観測が行われる→これをdataframeとする
## 'userid', 'bleid', 'rssi', 'time'をuserごとに時系列で並べたデータを発生させる！
## 観測がされるかされないかも乱数で1/2で決めるというのでもいいかもしれない．20m以下のビーコンは1/2，30m以下のビーコンは1/4．40m以下のビーコンは1/8，みたいな感じで実装できそうかも
## それぞれのrssiの発生させ方は，N, sigmaは所与，リンクが決まれば各ビーコンとの距離dも決まるのでdも所与，となったとき，N(d, sigma)の正規分布からd_estがバラついて発生する
## 発生したd_estからNを用いてrssiが取得されるので，これを観測データとしてdataframeに入れればいい，という流れか．

### とりあえずTS=T-1で読み替えている．TSは遷移回数で，これは今はTに等しい．いや実際にはT-2？？めんどいすぎる
# about T
## とりあえず観測できる数はT-2回＝8回
## 山野さん修論は遷移を観測しているが松永卒論では大山羽藤2017に倣って状態を観測するとして記述


s_time = round((end_time - start_time)/(T-2)) #???これでいい？ああインデックスめんどくさすぎる，，，
st = np.zeros((T-1, 1))
for ts in range(T-1):
    st[ts] = start_time + ts * s_time

TM = round((end_time - start_time) / m_time) ### TM=32, つまり32回の観測が発生するということ

mt = np.zeros((TM+1, 2)) 
for tm in range(TM+1):
    mt[tm, 0] = start_time + tm * m_time # 観測回数tm現在の時刻
    for ts in range(T-1): ## これは遷移回数？？？？？これでいいのか微妙だけどwwま，とりあえずです
        if mt[tm, 0] <= st[ts]:
            mt[tm, 1] = ts
            break
    mt[tm, 0] = start_time + tm * m_time

mt = pd.DataFrame(mt, columns=['time', 'timestep'])

print(st)
print(mt)

[[  0.]
 [ 20.]
 [ 40.]
 [ 60.]
 [ 80.]
 [100.]
 [120.]
 [140.]
 [160.]]
     time  timestep
0     0.0       0.0
1     5.0       1.0
2    10.0       1.0
3    15.0       1.0
4    20.0       1.0
5    25.0       2.0
6    30.0       2.0
7    35.0       2.0
8    40.0       2.0
9    45.0       3.0
10   50.0       3.0
11   55.0       3.0
12   60.0       3.0
13   65.0       4.0
14   70.0       4.0
15   75.0       4.0
16   80.0       4.0
17   85.0       5.0
18   90.0       5.0
19   95.0       5.0
20  100.0       5.0
21  105.0       6.0
22  110.0       6.0
23  115.0       6.0
24  120.0       6.0
25  125.0       7.0
26  130.0       7.0
27  135.0       7.0
28  140.0       7.0
29  145.0       8.0
30  150.0       8.0
31  155.0       8.0
32  160.0       8.0


いよいよ個人ごとの観測データログの作成に入る，，うわ，，，，，
尤度比や，accuracyを最尤法と多様体学習法とで比較し向上するという結果が示ればとてもいいのだけど，，

BLEデータ生成関数

In [34]:
def data_generator(x): 
    df_res_all = assignment(x) # 配分結果を読み込んで，BLE観測データを生成
    grouped = df_res_all.groupby('d') # odでグルーピング
    df_res_list = [group.reset_index(drop=True) for name, group in grouped]  

    ## 結果格納用
    generated_mdata = np.zeros(7) # 'userid', 'timestep', 'tt', 'bleid', 'rssi', 'o', 'd'

    for od in range(OD): # OD=len(df_demand) 
        olinkid = df_demand.loc[od, 'o']
        dlinkid = df_demand.loc[od, 'd']
        dfod = df_res_list[od]
        grouped2 = dfod.groupby('userid')
        dfod_res_list = [group2.reset_index(drop = True) for name, group2 in grouped2]

        for i in range(len(dfod_res_list)): # useridごと
            dfi = dfod_res_list[i]
            userid = int(dfi.loc[0, 'userid'])

            for t in range(len(mt)):
                if t == 0: # スタートリンクは観測なし
                    continue

                timestep = int(mt.loc[t, 'timestep']) # 1~8
                tt = int(mt.loc[t, 'time'])
                kid = int(dfi.loc[timestep, 'k']) # 時刻tt, timestep:timestepでの真の滞在リンク
                kidx = kid - 1

                if kid == 26: # 吸収リンクは観測なし
                    continue

                #print(f'---userid{userid}, timestep{timestep}, time{tt}でリンクは{kid}---')

                ## 理想的には，前後のkを参照して（前のkが25or 後ろのkが26なら良きように）動きの方向を識別し，tmにおける滞在位置をもう少し細かく指定する
                ## timstepを4分割しているわけなので，大まかに1/4ずつ進行しているという設定にすることは可能ではないか．
                ## その位置からビーコンまでの距離で観測のあるなしをやるのがいいと思うけど一旦は

                ## 本当はttじゃなくてtimestepでループ回してkはその外に置きたいけど一旦
                loc_o = link_loc_array[kidx][0] # oの座標のリスト
                loc_d = link_loc_array[kidx][1] # dの座標のリスト

                mid_loc = (loc_o + loc_d) / 2

                ## x_locを4等分したどこかにおくのがもう少し丁寧なので，後ほど実装する

                d_array_k = d_array[kidx, :] # 真リンクと各ビーコンとの距離：10m, 20m, 25m以下のビーコンを取得（設定）
                
                indices1 = np.argwhere((d_array_k <= 10))[:, 0].tolist() 
                indices2 = np.argwhere((d_array_k <= 20) & (d_array_k > 10))[:, 0].tolist()
                indices3 = np.argwhere((d_array_k <= 25) & (d_array_k > 20))[:, 0].tolist()

                column_indices_list1 = list(set(indices1))  # setで重複削除
                column_indices_list2 = list(set(indices2))  
                column_indices_list3 = list(set(indices3))  

                # print(f'10m以下のビーコンは{column_indices_list1}, 20m以下は{column_indices_list2}, 25m以下は{column_indices_list3}')
                
                ## それぞれ乱数使って3/4, 2/4, 1/4で捕捉されるとする（設定）
                for j1 in column_indices_list1:
                    ran = random.random()
                    if ran >= 0.25: # 確率3/4で観測される

                        ##### ここを，リンク上の厳密な位置として出したいが，，どうか #####
                        d = d_array[kidx, j1]
                        d_est = abs(np.random.normal(loc=d, scale=sigma)) # 正規分布逆関数
                        d_est = (d_est == 0) * 1 + (d_est != 0) * d_est 

                        print(f'user{userid}, timestep{timestep}, time{tt}, link{kid}, beacon{j1+1}, d={d}, d_est={d_est}')

                        rssi = rssi_max - 10 * N * np.log10(d_est) # 減衰関数逆関数
                        ## rssi_maxは超えないようにする
                        if rssi >= rssi_max:
                            rssi = rssi_max  
                        resi_array = np.array((userid, timestep, tt, j1+1, rssi, olinkid, dlinkid))
                        generated_mdata = np.vstack((generated_mdata, resi_array))
                
                # j2, j3のビーコンに対しても同様の処理
                for j2 in column_indices_list2:
                    ran = random.random()
                    if ran >= 0.5: # 確率2/4で観測される
                        d = d_array[kidx, j2]
                        d_est = abs(np.random.normal(loc=d, scale=sigma)) # 正規分布逆関数
                        d_est = (d_est == 0) * 1 + (d_est != 0) * d_est
                        print(f'user{userid}, timestep{timestep}, time{tt}, link{kid}, beacon{j2+1}, d={d}, d_est={d_est}')

                        rssi = rssi_max - 10 * N * np.log10(d_est) # 減衰関数逆関数
                        if rssi >= rssi_max:
                            rssi = rssi_max
                        resi_array = np.array((userid, timestep, tt, j2+1, rssi, olinkid, dlinkid))
                        generated_mdata = np.vstack((generated_mdata, resi_array))
                
                for j3 in column_indices_list3:
                    ran = random.random()
                    if ran >= 0.75: # 確率1/4で観測される
                        d = d_array[kidx, j3]
                        d_est = abs(np.random.normal(loc=d, scale=sigma)) # 正規分布逆関数
                        d_est = (d_est == 0) * 1 + (d_est != 0) * d_est
                        print(f'user{userid}, timestep{timestep}, time{tt}, link{kid}, beacon{j3+1}, d={d}, d_est={d_est}')

                        rssi = rssi_max - 10 * N * np.log10(d_est) # 減衰関数逆関数
                        if rssi >= rssi_max:
                            rssi = rssi_max
                        resi_array = np.array((userid, timestep, tt, j3+1, rssi, olinkid, dlinkid))
                        generated_mdata = np.vstack((generated_mdata, resi_array))
          
    generated_mdata = np.delete(generated_mdata, 0, axis = 0)
    df_generated_mdata = pd.DataFrame(generated_mdata, columns = ['userid', 'timestep', 'tt', 'bleid', 'rssi', 'o', 'd'])
    df_generated_mdata.to_csv('/Users/takahiromatsunaga/res2023/futago/generated_bledata_ver2.csv')

    return df_generated_mdata

In [35]:
print(data_generator(x))

user0, timestep1, time5, link1, beacon1, d=10.0, d_est=11.674194742980681
user0, timestep1, time10, link1, beacon1, d=10.0, d_est=9.89018396251115
user0, timestep1, time10, link1, beacon2, d=10.0, d_est=17.129236308386133
user0, timestep1, time15, link1, beacon1, d=10.0, d_est=11.243380649236771
user0, timestep1, time15, link1, beacon2, d=10.0, d_est=6.265401701863672
user0, timestep1, time15, link1, beacon6, d=22.360679774997898, d_est=22.32982674549053
user0, timestep1, time20, link1, beacon1, d=10.0, d_est=14.685212526979974
user0, timestep1, time20, link1, beacon5, d=22.360679774997898, d_est=21.47894190463869
user0, timestep2, time25, link2, beacon2, d=10.0, d_est=6.186484165124071
user0, timestep2, time25, link2, beacon3, d=10.0, d_est=20.178320669208276
user0, timestep2, time30, link2, beacon2, d=10.0, d_est=13.14419613667905
user0, timestep2, time30, link2, beacon3, d=10.0, d_est=12.477416409829202
user0, timestep2, time35, link2, beacon2, d=10.0, d_est=7.4395794472353955
user0

In [45]:
print(d_array)

[[ 10.          10.          30.          50.          22.36067977
   22.36067977  36.05551275  53.85164807  41.23105626  41.23105626
   50.          64.03124237  60.8276253   60.8276253   67.08203932
   78.10249676]
 [ 30.          10.          10.          30.          36.05551275
   22.36067977  22.36067977  36.05551275  50.          41.23105626
   41.23105626  50.          67.08203932  60.8276253   60.8276253
   67.08203932]
 [ 50.          30.          10.          10.          53.85164807
   36.05551275  22.36067977  22.36067977  64.03124237  50.
   41.23105626  41.23105626  78.10249676  67.08203932  60.8276253
   60.8276253 ]
 [ 22.36067977  22.36067977  36.05551275  53.85164807  10.
   10.          30.          50.          22.36067977  22.36067977
   36.05551275  53.85164807  41.23105626  41.23105626  50.
   64.03124237]
 [ 36.05551275  22.36067977  22.36067977  36.05551275  30.
   10.          10.          30.          36.05551275  22.36067977
   22.36067977  36.05551275  50.